# 波の屈折

In [ ]:
import numpy as np
from math import cos, pi
from matplotlib import pyplot, animation, rc
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

In [ ]:
s = 1.0           # 波の伝わる速さ
length = 1.0      # 系の長さ
time = 1.2        # シミュレーション時間
nx = 100          # 空間メッシュ数
dx = length/nx    # 空間刻み幅
dt = 0.5*dx       # 時間刻み幅
nt = int(time/dt) # 時間刻み数
a = s*dt/dx       # α

In [ ]:
# 初期条件
x, y = np.mgrid[0:1:dx,0:1:dx]
u0 = np.zeros((nx,nx)) # 初期変位
v0 = np.zeros((nx,nx)) # 初期速度

In [ ]:
ax = np.zeros((nx,nx))
ax[:,:] = a
for i in range(nx):
    for j in range(nx):
        if x[i,j] > 0.8*y[i,j]+0.2:
            ax[i,j] = 0.6*a

In [ ]:
# 強制振動
f = 0.1 # 周期
def oscillate(i):
    return 0.2*cos((2*pi/f)*dt*i)

In [ ]:
# シミュレーション
u = np.zeros((nt,nx,nx))
u[0,:,:] = u0[:,:]
u[0,0,:] = oscillate(0)
u[1,1:-1,1:-1] = u[0,1:-1,1:-1]+dt*v0[1:-1,1:-1] \
       +(ax[1:-1,1:-1]**2/2)*(u[0,0:-2,1:-1]+u[0,2:nx,1:-1] \
       +u[0,1:-1,0:-2]+u[0,1:-1,2:nx]-4*u[0,1:-1,1:-1])
u[1,0,:] = oscillate(1)
for i in range(2,nt):
    u[i,1:-1,1:-1] = 2*u[i-1,1:-1,1:-1]-u[i-2,1:-1,1:-1] \
        +ax[1:-1,1:-1]**2*(u[i-1,0:-2,1:-1]+u[i-1,2:nx,1:-1] \
        +u[i-1,1:-1,0:-2]+u[i-1,1:-1,2:nx]-4*u[i-1,1:-1,1:-1])
    u[i,0,:] = oscillate(i)

# 結果の描画
def update(i):
    ax.clear()
    ax.plot_wireframe(x, y, u[i], linewidth=0.5)
    ax.set_zlim(-0.5,0.5)
    ax.set_title('step=' + str(i))
fig = pyplot.figure()
ax = fig.add_subplot(projection='3d')
movie = animation.FuncAnimation(fig, update, frames=nt, interval=50)
rc('animation', html='jshtml')
movie

In [ ]:
# 等高線プロット
def update(i):
    pyplot.cla()
    pyplot.contourf(x, y, u[i])
    pyplot.plot([0.2,1.0],[0.0,1.0],"r")
    pyplot.title('step=' + str(i))
    pyplot.gca().set_aspect('equal');
fig = pyplot.figure()
movie = animation.FuncAnimation(fig, update, frames=nt, interval=50)
rc('animation', html='jshtml')
movie